In [1]:
#Starter notebook taken from hiromu
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [16]:
tourney_result = pd.read_csv('WDataFiles_Stage1/WNCAATourneyCompactResults.csv')
tourney_seed = pd.read_csv('WDataFiles_Stage1/WNCAATourneySeeds.csv')
tourney_seed

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272
...,...,...,...
1403,2019,Z12,3416
1404,2019,Z13,3195
1405,2019,Z14,3200
1406,2019,Z15,3340


In [17]:
# deleting unnecessary columns
tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
tourney_result

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208
...,...,...,...
1381,2019,3124,3234
1382,2019,3323,3390
1383,2019,3124,3332
1384,2019,3323,3163


In [18]:
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'WSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'LSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,X02,X15
1,1998,3112,3365,W03,W14
2,1998,3163,3193,W02,W15
3,1998,3198,3266,Y07,Y10
4,1998,3203,3208,W10,W07
...,...,...,...,...,...
1381,2019,3124,3234,Y01,Y02
1382,2019,3323,3390,X01,X02
1383,2019,3124,3332,Y01,Z02
1384,2019,3323,3163,X01,W02


In [21]:
def get_seed(x):
    return int(x[1:3])

tourney_result['WSeed'] = tourney_result['WSeed'].map(lambda x: get_seed(x))
tourney_result['LSeed'] = tourney_result['LSeed'].map(lambda x: get_seed(x))
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,2,15
1,1998,3112,3365,3,14
2,1998,3163,3193,2,15
3,1998,3198,3266,7,10
4,1998,3203,3208,10,7
...,...,...,...,...,...
1381,2019,3124,3234,1,2
1382,2019,3323,3390,1,2
1383,2019,3124,3332,1,2
1384,2019,3323,3163,1,2


In [22]:
season_result = pd.read_csv('WDataFiles_Stage1/WRegularSeasonCompactResults.csv')

In [23]:
season_win_result = season_result[['Season', 'WTeamID', 'WScore']]
season_lose_result = season_result[['Season', 'LTeamID', 'LScore']]
season_win_result.rename(columns={'WTeamID':'TeamID', 'WScore':'Score'}, inplace=True)
season_lose_result.rename(columns={'LTeamID':'TeamID', 'LScore':'Score'}, inplace=True)
season_result = pd.concat((season_win_result, season_lose_result)).reset_index(drop=True)
season_result

/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Season,TeamID,Score
0,1998,3104,91
1,1998,3163,87
2,1998,3222,66
3,1998,3307,69
4,1998,3349,115
...,...,...,...
214019,2019,3251,49
214020,2019,3179,79
214021,2019,3335,54
214022,2019,3144,45


In [24]:
season_score = season_result.groupby(['Season', 'TeamID'])['Score'].sum().reset_index()
season_score

,Season,TeamID,Score
0,1998,3102,1375
1,1998,3103,2008
2,1998,3104,2297
3,1998,3106,1286
4,1998,3108,1560
...,...,...,...
7350,2019,3462,1765
7351,2019,3463,1837
7352,2019,3464,1976
7353,2019,3465,1905


In [25]:
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'WScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'LScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WScoreT,LScoreT
0,1998,3104,3422,2,15,2297,2010
1,1998,3112,3365,3,14,2116,2144
2,1998,3163,3193,2,15,2696,1950
3,1998,3198,3266,7,10,2337,1907
4,1998,3203,3208,10,7,1818,1996
...,...,...,...,...,...,...,...
1381,2019,3124,3234,1,2,2575,2555
1382,2019,3323,3390,1,2,2938,2393
1383,2019,3124,3332,1,2,2575,2725
1384,2019,3323,3163,1,2,2938,2726


In [26]:
tourney_win_result = tourney_result.drop(['Season', 'WTeamID', 'LTeamID'], axis=1)
tourney_win_result.rename(columns={'WSeed':'Seed1', 'LSeed':'Seed2', 'WScoreT':'ScoreT1', 'LScoreT':'ScoreT2'}, inplace=True)
tourney_win_result

,Seed1,Seed2,ScoreT1,ScoreT2
0,2,15,2297,2010
1,3,14,2116,2144
2,2,15,2696,1950
3,7,10,2337,1907
4,10,7,1818,1996
...,...,...,...,...
1381,1,2,2575,2555
1382,1,2,2938,2393
1383,1,2,2575,2725
1384,1,2,2938,2726
